# MELON CHART PREDICTION PROJECT

## I. Set up


In [ ]:
import pandas as pd
import json as js 
import numpy as np
import re
import requests 
from bs4 import BeautifulSoup
from time import sleep
#from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager

## 1. Load Melon datasets

In [ ]:
curr_path = '/content/drive/MyDrive/Self-study/ML/melon/'
train = pd.read_json(curr_path + 'melon_data/train.json')
#val = pd.read_json(curr_path + 'melon_data/val.json')
#test = pd.read_json(curr_path + 'melon_data/test.json')
song_meta = pd.read_json(curr_path + 'melon_data/song_meta.json')

NameError: ignored

In [ ]:
genre_file = open(curr_path + 'melon_data/genre_gn_all.json')

genre_js = js.load(genre_file)
genre = pd.DataFrame(genre_js, index = [['genre']]).T

In [ ]:
train.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


In [ ]:
song_meta.head(3)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2


In [ ]:
genre.head(3)

,genre
GN0100,발라드
GN0101,세부장르전체
GN0102,'80


## 2. Scrape Melon chart data

highest chart ranking
add metadata:
- artist gender 
- playlist tag
- weighted like, share, count, etc of the songs?
- artist like counts
- artist debut date
- band or not?

Github link for melon api: https://github.com/ko28/melon-api

- Some issue dates are missing. Mostly old songs
- Some songIds are missing
Only get songs from 2000 to 2022 because older songs may not be a good training example for new songs.
- Domestic comprehensive for 2000s

In [ ]:
#get the min and max year from songmeta
issue_year_list = []
for date in song_meta['issue_date']:
    if date != 0:
        issue_year_list.append(int(str(date)[:4]))
    else:
        issue_year_list.append(np.nan)

song_meta['issue_year'] = issue_year_list

In [ ]:
song_meta = song_meta[song_meta['issue_year'] >= 2017]

In [ ]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,issue_year
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,2018.0
6,"[GN1601, GN1602, GN1614]",20170208,Sibelius: Symphony No. 2; Tapiola; Valse triste,10038397,"[162097, 100237]",Sibelius : Valse Triste Op.44 (시벨리우스 : 슬픈 왈츠 작...,[GN1600],"[Herbert Blomstedt, San Francisco Symphony]",6,2017.0
7,"[GN1601, GN1602, GN1614]",20190315,Celebrating John Williams (Live At Walt Disney...,10261160,"[105210, 261542]",Superman March (From &#34;Superman&#34; / Live...,[GN1600],"[Los Angeles Philharmonic Orchestra, Gustavo D...",7,2019.0
9,"[GN0105, GN0101]",20170320,Pastel Reflection,10047088,[753752],"사랑, 그대라는 멜로디",[GN0100],[진호],9,2017.0
10,[GN1201],20170407,Luv.Loops,10053652,[1625859],Hi (Heyoo),[GN1200],[Miraa.],10,2017.0
...,...,...,...,...,...,...,...,...,...,...
707976,"[GN1301, GN1302]",20181018,Colors Compilation,2692383,[967093],Everybody (Just Bounce),[GN1300],[Vbnd],707976,2018.0
707977,"[GN1601, GN1606]",20181123,Three Great Danish Women Pianists,10225022,"[2402744, 2402745, 2402746]",Beethoven : Bagatelle In A Minor WoO.59 `Fur E...,[GN1600],"[France Ellegaard, Galina Werschensk, Johanne ...",707977,2018.0
707978,"[GN0908, GN0901]",20191025,My Blood,10343254,[100377],My Blood,[GN0900],[Westlife],707978,2019.0
707980,"[GN0401, GN0403]",20171202,Mirrorball,10116357,[924416],Mirrorball (Feat. YunB),[GN0400],[SUMIN (수민)],707980,2017.0


In [ ]:
def get_song_id(temp_html):
    try: 
        target = temp_html.find('div', {'class': 'ellipsis rank01'}).a['href']
        pattern = 'javascript:melon.play.playSong\(\'19080101\',\'(\d*)\'\);'
        return re.findall(pattern, target)[0]
    except: 
        return np.nan

def get_artist_id(temp_html):
    try: 
        target = temp_html.find('div', {'class': 'ellipsis rank02'}).a['href']
        pattern = r'javascript:melon.link.goArtistDetail\(\'(\d*)\'\);'
        return re.findall(pattern, target)[0]
    except:
        return np.nan

def get_alb_id(temp_html):
    try:
        target = temp_html.find('div', {'class': 'ellipsis rank03'}).a['href']
        pattern = r'javascript:melon.link.goAlbumDetail\(\'(\d*)\'\);'
        return re.findall(pattern, target)[0]
    except:
        return np.nan

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.melon.com/")

In [ ]:
main_url = 'https://www.melon.com/chart/search/list.htm?chartType=WE&age={age}&year={year}&mon={mon}&day={day}&classCd={classCd}&startDay={startDay}&endDay={endDay}&moved=Y'

def save_chart(age, year, mon, day, classCd, startDay, endDay):
    global main_url
    sub_url = main_url.format(age = age, year = year, mon = mon, day = day, classCd = classCd, startDay = startDay, endDay = endDay)
    driver.get(sub_url)
    bs = BeautifulSoup(driver.page_source, 'html.parser')
    chart_list = []
    bs_list = bs.tbody.find_all('tr',{'class': 'lst50'})
    bs_list += bs.tbody.find_all('tr',{'class': 'lst100'})
    file_path = curr_path + 'melon_data/chart/{startDay}.csv'

    for temp_html in bs_list:
        temp_dict = {}

        #temp_dict['rank']
        try: 
            temp_dict['rank'] = temp_html.find('span', {'class': 'rank top'}).text
        except: 
            temp_dict['rank'] = temp_html.find('span', {'class': 'rank'}).text

        #temp_dict['song_name']
        try:
            temp_dict['song_name'] = temp_html.find('div', {'class': 'wrap_song_info'}).a.text
        except: 
            temp_dict['song_name'] = np.nan

        #temp_dict['song_id']
        temp_dict['song_id'] = get_song_id(temp_html)

        #temp_dict['artist_name']
        try:
            temp_dict['artist_name'] = temp_html.find('div', {'class': 'ellipsis rank02'}).a.text.strip()
        except: 
            temp_dict['artist_name'] = np.nan

        #temp_dict['artist_id']
        temp_dict['artist_id'] = get_artist_id(temp_html)

        #temp_dict['alb_name']
        try:
            temp_dict['alb_name'] = temp_html.find('div', {'class': 'ellipsis rank03'}).a.text.strip()
        except:
            temp_dict['alb_name'] = np.nan

        #temp_dict['alb_id']
        temp_dict['alb_id'] = get_alb_id(temp_html)

        chart_list.append(temp_dict)
    chart_df = pd.DataFrame(chart_list)
    chart_df.to_csv(file_path.format(startDay = startDay))

In [ ]:
date_df = pd.read_csv(curr_path + 'melon_data/date.csv')

In [ ]:
date_df

,age,year,mon,day,classCd,startDay,endDay
0,2000,2000,1,20000102%5E20000108,KPOP,20000102,20000108
1,2000,2000,1,20000109%5E20000115,KPOP,20000109,20000115
2,2000,2000,1,20000116%5E20000122,KPOP,20000116,20000122
3,2000,2000,1,20000123%5E20000129,KPOP,20000123,20000129
4,2000,2000,2,20000130%5E20000205,KPOP,20000130,20000205
...,...,...,...,...,...,...,...
1164,2020,2022,4,20220425%5E20220501,GN0000,20220425,20220501
1165,2020,2022,5,20220502%5E20220508,GN0000,20220502,20220508
1166,2020,2022,5,20220509%5E20220515,GN0000,20220509,20220515
1167,2020,2022,5,20220516%5E20220522,GN0000,20220516,20220522


In [ ]:
for row_idx, row_series in date_df.iloc.iterrows():
    age = row_series[0]
    year = row_series[1]
    mon = row_series[2]
    day = row_series[3]
    classCd = row_series[4]
    startDay = row_series[5]
    endDay = row_series[6]
    save_chart(age, year, mon, day, classCd, startDay, endDay)
    sleep(2)

- class = 'rank'
- class="ellipsis rank01" => song title
- class="ellipsis rank02" => artist name (goArtistDetail -> id)
- data-song-no => songId
- class="ellipsis rank03" => album name (goAlbumDetail -> id)

(also have info about music video)

## 3. Add two variables: `highest chart ranking` and `top50` (indicate whether a song is in top 100 weekly or not)

In [ ]:
date_df = pd.read_csv(curr_path + 'melon_data/date.csv')

In [ ]:
startDay_list = []

for row_idx, row_series in date_df.iterrows():
    startDay_list.append(row_series[5])

startDay_list = startDay_list[:-1]

In [ ]:
file_path_list = [curr_path + 'melon_data/chart/{startDay}.csv'.format(startDay = day) for day in startDay_list]

In [ ]:
songs_list = []

for file_path in file_path_list:
    df = pd.read_csv(file_path)
    try: 
        songs_list += list(df['song_id'])
    except: 
        print(file_path)

In [ ]:
unique_songs_set = set(songs_list)

In [ ]:
istop = []
for song_id in song_meta['id']:
    if song_id not in unique_songs_set:
        istop.append(0)
    else:
        istop.append(1)

song_meta['istop'] = istop

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
song_meta.iloc[0,5]

'Solsbury Hill (Remastered 2002)'

In [ ]:
song_meta['istop'].value_counts()

0    187044
Name: istop, dtype: int64

In [ ]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,issue_year,istop
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,2018.0,0
6,"[GN1601, GN1602, GN1614]",20170208,Sibelius: Symphony No. 2; Tapiola; Valse triste,10038397,"[162097, 100237]",Sibelius : Valse Triste Op.44 (시벨리우스 : 슬픈 왈츠 작...,[GN1600],"[Herbert Blomstedt, San Francisco Symphony]",6,2017.0,0
7,"[GN1601, GN1602, GN1614]",20190315,Celebrating John Williams (Live At Walt Disney...,10261160,"[105210, 261542]",Superman March (From &#34;Superman&#34; / Live...,[GN1600],"[Los Angeles Philharmonic Orchestra, Gustavo D...",7,2019.0,0
9,"[GN0105, GN0101]",20170320,Pastel Reflection,10047088,[753752],"사랑, 그대라는 멜로디",[GN0100],[진호],9,2017.0,0
10,[GN1201],20170407,Luv.Loops,10053652,[1625859],Hi (Heyoo),[GN1200],[Miraa.],10,2017.0,0
...,...,...,...,...,...,...,...,...,...,...,...
707976,"[GN1301, GN1302]",20181018,Colors Compilation,2692383,[967093],Everybody (Just Bounce),[GN1300],[Vbnd],707976,2018.0,0
707977,"[GN1601, GN1606]",20181123,Three Great Danish Women Pianists,10225022,"[2402744, 2402745, 2402746]",Beethoven : Bagatelle In A Minor WoO.59 `Fur E...,[GN1600],"[France Ellegaard, Galina Werschensk, Johanne ...",707977,2018.0,0
707978,"[GN0908, GN0901]",20191025,My Blood,10343254,[100377],My Blood,[GN0900],[Westlife],707978,2019.0,0
707980,"[GN0401, GN0403]",20171202,Mirrorball,10116357,[924416],Mirrorball (Feat. YunB),[GN0400],[SUMIN (수민)],707980,2017.0,0


## 4. Add additional metadata

Remove songs with various artist

SHOULD I ADD ALL INFO AS POSSIBLE

In [ ]:
idx = 0
idx_lst = []
for id in song_meta['artist_id_basket']:
    if len(id) < 2:
        idx_lst.append(idx)
    idx+=1

In [ ]:
song_meta_main = song_meta.iloc[idx_lst,:]
song_meta_main.to_csv(curr_path + 'melon_data/song_meta_main.csv')

In [ ]:
song_meta_main = pd.read_csv(curr_path + 'melon_data/song_meta_main.csv')

In [ ]:
song_meta_main

,Unnamed: 0,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,istop
0,0,['GN0901'],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,['GN0900'],['Various Artists'],0,0
1,1,"['GN1601', 'GN1606']",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",['GN1600'],['Murray Perahia'],1,0
2,2,['GN0901'],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),['GN0900'],['Peter Gabriel'],2,0
3,3,"['GN1102', 'GN1101']",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,['GN1100'],['Matoma'],3,0
4,4,"['GN1802', 'GN1801']",20110824,그남자 그여자,2008470,[560160],그남자 그여자,['GN1800'],['Jude Law'],4,0
...,...,...,...,...,...,...,...,...,...,...,...
658433,707984,['GN2001'],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,['GN2000'],['Fela Kuti'],707984,0
658434,707985,['GN0901'],19860000,True Colors,44141,[11837],Change Of Heart,['GN0900'],['Cyndi Lauper'],707985,0
658435,707986,"['GN0105', 'GN0101']",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,['GN0100'],['윤종신'],707986,0
658436,707987,"['GN1807', 'GN1801']",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,['GN1800'],['Nature Piano'],707987,0


In [ ]:
song_meta_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707989 entries, 0 to 707988
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Unnamed: 0              707989 non-null  int64 
 1   song_gn_dtl_gnr_basket  707989 non-null  object
 2   issue_date              707989 non-null  int64 
 3   album_name              707982 non-null  object
 4   album_id                707989 non-null  int64 
 5   artist_id_basket        707989 non-null  object
 6   song_name               707987 non-null  object
 7   song_gn_gnr_basket      707989 non-null  object
 8   artist_name_basket      707989 non-null  object
 9   id                      707989 non-null  int64 
 10  istop                   707989 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 59.4+ MB


In [ ]:
main_url = 'https://www.melon.com/artist/detail.htm?artistId={artist_id}'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15'
headers = {'User-Agent': user_agent}

def get_artist_metadata(artist_id):
    sub_url = main_url.format(artist_id = artist_id)
    r = requests.get(sub_url, headers = headers)
    bs = BeautifulSoup(r.text)

    temp_dict = {'artist_id': artist_id}

    ####
    bs_info1 = bs.find('div', {'class': 'section_atistinfo03'})

    pattern = r'(\d{4}\.\d{2}(?:\.\d{2})?)'
    try:    
        target = bs_info1.find('dl', {'class': 'list_define clfix'}).text
        temp_dict['debut_date'] = re.findall(pattern, target)[0]
    except:
        temp_dict['debut_date'] = np.nan

    pattern =  r'유형\n(\w*)[\s\S]*\|(\w*)'
    try: 
        temp_dict['artist_type'], temp_dict['artist_gender'] = re.findall(pattern, target)[0]
    except:
        temp_dict['artist_type'], temp_dict['artist_gender'] = np.nan, np.nan

    ####
    bs_info2 = bs.find('div', {'class': 'section_atistinfo04'})

    pattern = r'국적\n(\w*)'
    try: 
        target = bs_info2.find('dl', {'class': 'list_define clfix'}).text
        temp_dict['artist_nationality'] = re.findall(pattern, target)[0]
    except: 
        temp_dict['artist_nationality'] = np.nan
    
    pattern = r'혈액형\n(\w*)'
    try:
        temp_dict['artist_bloodtype'] = re.findall(pattern,target)
    except:
        temp_dict['artist_bloodtype'] = np.nan

    ####
    bs_info3 = bs.find('div', {'class': 'section_atistinfo05'})
    
    pattern = r'(http(?:s)?://(?:www.)?youtube.com/\w*)'
    try:
        target = bs_info3.text
        temp_dict['artist_yt'] = re.findall(pattern, target)[0]
    except:
        temp_dict['artist_yt'] = np.nan
    
    try:
        target = bs_info3.find_all('button')    
        pattern = r'http(?:s)?://twitter.com/\w*'
        temp_dict['artist_tw'] = re.findall(pattern, target[0]['onclick'])[0]
    except:
        temp_dict['artist_tw'] = np.nan
    
    try:
        pattern = r'http(?:s)?://www.facebook.com/\w*'
        temp_dict['artist_fb'] = re.findall(pattern, target[1]['onclick'])[0]  
    except:
        temp_dict['artist_fb'] = np.nan 

    return temp_dict 

In [ ]:
artist_metadata_dict = []

for artist_id in song_meta_main['artist_id_basket'][0:100000]:
    try:
        temp_dict = get_artist_metadata(artist_id[0])
        artist_metadata_dict.append(temp_dict)
    except:
        print(artist_id)

[222305]
[247618]
[314367]
[104039]


### Add artist metadata

In [ ]:
main_url = 'https://www.melon.com/artist/detail.htm?artistId={artist_id}'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15'
headers = {'User-Agent': user_agent}

def get_artist_metadata(artist_id):
    sub_url = main_url.format(artist_id = artist_id)
    r = requests.get(sub_url, headers = headers)
    bs = BeautifulSoup(r.text)

    temp_dict = {}
    
    ####
    bs_info1 = bs.find('div', {'class': 'section_atistinfo03'})

    pattern = r'(\d{4}(?:\.\d{2}\.\d{2})?)'
    try:    
        target = bs_info1.find('dl', {'class': 'list_define clfix'}).text
        temp_dict['debut_date'] = re.findall(pattern, target)[0]
    except:
        temp_dict['debut_date'] = np.nan

    pattern =  r'유형\n(\w*)[\s\S]*\|(\w*)'
    try: 
        temp_dict['artist_type'], temp_dict['artist_gender'] = re.findall(pattern, target)[0]
    except:
        temp_dict['artist_type'], temp_dict['artist_gender'] = np.nan, np.nan

    ####
    bs_info2 = bs.find('div', {'class': 'section_atistinfo04'})

    pattern = r'국적\n(\w*)'
    try: 
        target = bs_info2.find('dl', {'class': 'list_define clfix'}).text
        temp_dict['artist_nationality'] = re.findall(pattern, target)[0]
    except: 
        temp_dict['artist_nationality'] = np.nan
    
    # pattern = r'혈액형\n(\w*)'
    # try:
    #     temp_dict['artist_bloodtype'] = re.findall(pattern,target)
    # except:
    #     temp_dict['artist_bloodtype'] = np.nan

    # ####
    # bs_info3 = bs.find('div', {'class': 'section_atistinfo05'})
    
    # pattern = r'(http(?:s)?://(?:www.)?youtube.com/\w*)'
    # try:
    #     target = bs_info3.text
    #     temp_dict['artist_yt'] = re.findall(pattern, target)[0]
    # except:
    #     temp_dict['artist_yt'] = np.nan
    
    # try:
    #     target = bs_info3.find_all('button')    
    #     pattern = r'http(?:s)?://twitter.com/\w*'
    #     temp_dict['artist_tw'] = re.findall(pattern, target[0]['onclick'])[0]
    # except:
    #     temp_dict['artist_tw'] = np.nan
    
    # try:
    #     pattern = r'http(?:s)?://www.facebook.com/\w*'
    #     temp_dict['artist_fb'] = re.findall(pattern, target[1]['onclick'])[0]  
    # except:
    #     temp_dict['artist_fb'] = np.nan 

    return temp_dict 

In [ ]:
artist_metadata = {}
idx_lst = list(range(1000,11000,1000))
for idx in range(len(idx_lst)-1):
    for artist_id in main_df['artist_id'][idx_lst[idx]:idx_lst[idx+1]]:
        if artist_id in artist_metadata:
            continue
        artist_metadata[artist_id] = get_artist_metadata(artist_id)
    sleep(60)

In [ ]:
artist_df = pd.DataFrame(artist_metadata2).T

In [ ]:
artist_df.to_csv(curr_path + 'melon_data/artist2')

### Add artist like

In [ ]:
def get_song_like(artist_id):
    main_url = 'https://www.melon.com/artist/getArtistFanNTemper.json?artistId={artist_id}'
    sub_url = main_url.format(artist_id = artist_id)
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15'
    headers = {'User-Agent': user_agent}
    r = requests.get(sub_url, headers = headers).json()
    return r['fanInfo']['SUMMCNT']

In [ ]:
artist_likes = {}
idx_lst = list(range(0,11000,1000))
for idx in range(len(idx_lst)-1):
    for artist_id in main_df['artist_id'][idx_lst[idx]:idx_lst[idx+1]]:
        if artist_id in artist_likes:
            continue
        artist_likes[artist_id] = get_song_like(artist_id)
    time.sleep(60)

In [ ]:
main_url = 'https://www.melon.com/artist/getArtistFanNTemper.json?artistId=3361'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15'
headers = {'User-Agent': user_agent}
r = requests.get(main_url, headers = headers).json()
r